# 1. Domača naloga: Uvoz in čiščenje podatkov

In [36]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import os
import numpy as np
import lxml
from io import StringIO

## List of countries and sependencies and their capitals in native languages

In [40]:
# URL strani z iskanjem tabele
url1 = 'https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20countries%20and%20dependencies%20and%20their%20capitals%20in%20native%20languages%20-%20Wikipedia.html'

# Pridobimo vsebino strani
vsebina1 = requests.get(url1)
soup1 = BeautifulSoup(vsebina1.content, 'html.parser')

# Poiščemo vse tabele na strani
tabele1 = soup1.find_all('table', {'class': 'wikitable'})

# Inicializiramo seznam za vse podatke
vsi_podatki1 = []

def ocisti(drzava_ali_mesto):
    if isinstance(drzava_ali_mesto, str):
        drzava_ali_mesto = re.split(r'\(',drzava_ali_mesto)[0]
        return drzava_ali_mesto.strip()
    return drzava_ali_mesto

# Obdelamo vsako tabelo posebej
for tabela in tabele1:
    vrstice = tabela.find_all('tr')[1:]  # Preskočimo prvo vrstico z glavo
    for vr in vrstice:
        stolpci = vr.find_all('td')
        # Preskočimo vrstice brez dovolj stolpcev
        if len(stolpci) < 2:
            continue
        # Odstranimo oznake <sup>
        for sup in vr.find_all('sup'):
            sup.decompose()
        # Pridobimo državo in prestolnico
        drzava = stolpci[0].text.strip()
        gl_mesto = stolpci[1].text.strip()
        # Dodamo le veljavne vrstice
        if drzava and gl_mesto:
            vsi_podatki1.append([ocisti(drzava), ocisti(gl_mesto)])

# Ustvarimo pandas DataFrame
glavna_mesta = pd.DataFrame(vsi_podatki1, columns=['drzava', 'glavno mesto'])
# Shranimo v CSV
glavna_mesta.to_csv(r'C:\Users\Aneja\Documents\pavp\glavna_mesta.csv', index=False)
glavna_mesta

,drzava,glavno mesto
0,Afghanistan,Kabul
1,Albania,Tirana
2,Algeria,Algiers
3,American Samoa,Pago Pago
4,Andorra,Andorra la Vella
...,...,...
239,Vietnam,Hanoi
240,Wallis and Futuna,Mata Utu
241,Yemen,Sana'a
242,Zambia,Lusaka


## Population Growth Rates in Urban areas and Capital cities

In [177]:
# URL datoteke
url2 = "https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/SYB61_253_Population%20Growth%20Rates%20in%20Urban%20areas%20and%20Capital%20cities.csv"

# Branje CSV datoteke
tabela2 = pd.read_csv(url2)

#odstranimo header vrstico
urejena = tabela2.iloc[1:]
#nastavimo stolpce
urejena.columns = tabela2.iloc[0]
#ponastavimo indekse
urejena = urejena.reset_index(drop=True)
#preimenujemo stolpce
urejena.rename(columns={'Year': 'leto','Series': 'spremenljivka','Value': 'vrednost'},inplace=True)
urejena = urejena.rename(columns={urejena.columns[1]: 'enota'})

filtrirana = urejena[urejena['spremenljivka'] != 'Rural population (percent growth rate per annum)']
filtrirana = filtrirana[filtrirana['spremenljivka'] != 'Urban population (percent growth rate per annum)']
filtrirana = filtrirana[filtrirana['spremenljivka'] != 'Capital city population (as a percentage of total urban population)']

populacija = pd.DataFrame(filtrirana, columns=['enota', 'leto','spremenljivka','vrednost'])
populacija = populacija.iloc[4:]
populacija = populacija.reset_index(drop=True)
populacija['vrednost'] = populacija['vrednost'].fillna(0)
populacija.to_csv(r'C:\Users\Aneja\Documents\pavp\populacija.csv', index=False)
populacija


,enota,leto,spremenljivka,vrednost
0,Africa,2005,Urban population (percent),36.9
1,Africa,2010,Urban population (percent),38.9
2,Africa,2015,Urban population (percent),41.2
3,Africa,2018,Urban population (percent),42.5
4,Northern Africa,2005,Urban population (percent),49.3
...,...,...,...,...
2149,Zimbabwe,2015,Urban population (percent),32.4
2150,Zimbabwe,2015,Capital city population (thousands),"1,500"
2151,Zimbabwe,2015,Capital city population (as a percentage of to...,9.5
2152,Zimbabwe,2018,Urban population (percent),32.2


## List of Academy Award-winning films

In [130]:
url3 = 'https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20Academy%20Award%e2%80%93winning%20films%20-%20Wikipedia.html'

# Pridobimo vsebino strani
tabele3 = pd.read_html(url3)
df = tabele3[0]
df = df.dropna()
# Pretvorba stolpcev Awards in Nominations v številske vrednosti
df["Awards"] = pd.to_numeric(df["Awards"], errors="coerce")
df["Nominations"] = pd.to_numeric(df["Nominations"], errors="coerce")
df = df[df['Nominations'] != 0]
#ustvarimo tabelo s true in tabelo s false za oskarje
df_true = df.copy()
df_false = df.copy()
df_true['oskar'] = True
df_false['oskar'] = False
df_true['stevilo'] = df_true['Awards']
df_false['stevilo'] = df_false['Nominations'] - df_false['Awards']
df_final = pd.concat([df_true, df_false], ignore_index=True)

filmi = df_final[['Film','Year','oskar','stevilo']]


# Sortiramo DataFrame po filmu in potem po sort_key
filmi = filmi.sort_values(by='Film', ascending=True)

filmi.rename(columns={'Film': 'naslov', 'Year': 'leto'},inplace=True)
filmi['leto'] = filmi['leto'].str.split('/').str[0]

# Odstranimo stolpec 'sort_key', saj ni več potreben
filmi = filmi.reset_index(drop=True)
filmi.to_csv(r'C:\Users\Aneja\Documents\pavp\filmi.csv', index=False)
filmi


,naslov,leto,oskar,stevilo
0,12 Years a Slave,2013,False,6.0
1,12 Years a Slave,2013,True,3.0
2,1917,2019,True,3.0
3,1917,2019,False,7.0
4,20 Days in Mariupol,2023,False,0.0
...,...,...,...,...
2713,Zootopia,2016,True,1.0
2714,Zorba the Greek (Alexis Zorbas),1964,True,3.0
2715,Zorba the Greek (Alexis Zorbas),1964,False,4.0
2716,tom thumb,1958,True,1.0


## List of Academy Award-winning foreign-language films

In [294]:
url4 = 'https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20Academy%20Award%e2%80%93winning%20foreign-language%20films%20-%20Wikipedia.html'
tabela4 = pd.read_html(url4)[0]

# Odstranimo vsebino v oklepajih (vključno z oklepaji) za vse celice v tabeli
tabela4 = tabela4.map(lambda x: x.split('(')[0].strip() if isinstance(x, str) else x)
tabela4 = tabela4.map(lambda x: x.split('[')[0].strip() if isinstance(x, str) else x)

vse_kategorije = ['Writing','Costume Design','Documentary Feature','Music','Film Editing','Cinematography','Art Direction','Actor in a Leading Role','Sound Editing','Makeup','Picture','Director','Original Screenplay','Production Design','Visual Effects','Sound','Animated Feature','Documentary Feature']

def razdeli_vrstico(vrstica, kategorije):
    rezultat = []
    for kategorija in kategorije:
        if kategorija in vrstica['Category']:
            nova_vrstica = vrstica.copy()
            nova_vrstica['Category'] = kategorija
            rezultat.append(nova_vrstica)
    return rezultat

def razdeli_po_jeziku(vrstica):
    rezultat = []
    jeziki = vrstica['Language(s)'].split(',')
    for jezik in jeziki:
        nova_vrstica = vrstica.copy()
        nova_vrstica['Language(s)'] = jezik.strip()
        rezultat.append(nova_vrstica)
    return rezultat

#ustvarimo novo tabelo z razširjenimi vrsticami na podlagi kategorij
razsirjene_vrstice = []
for _, vrstica in tabela4.iterrows():
    razsirjene_vrstice.extend(razdeli_vrstico(vrstica, vse_kategorije))

razsirjena_po_kategorijah = pd.DataFrame(razsirjene_vrstice).reset_index(drop=True)

#ustvarimo novo tabelo z razširjenimi vrsticami na podlagi jezikov
razsirjene_vrstice2 = []
for _,vrstica in razsirjena_po_kategorijah.iterrows():
    razsirjene_vrstice2.extend(razdeli_po_jeziku(vrstica))

tuji_filmi1 = pd.DataFrame(razsirjene_vrstice2).reset_index(drop=True)
tuji_filmi1.rename(columns={'Film title used in nomination': 'naslov_nom', 'Original title': 'naslov_orig', 'Year (Ceremony)': 'leto', 'Category': 'kategorija', 'Country':'drzava', 'Language(s)':'jezik'},inplace=True)

tuji_filmi = tuji_filmi1[['naslov_nom','naslov_orig','leto','kategorija','drzava','jezik']]

#stolpec zap_st_podelitev manjka

potrebno_dodati = ['Shoeshine','Gate of Hell','Parasite','The Artist','Roma']

tuji_filmi.to_csv(r'C:\Users\Aneja\Documents\pavp\tuji_filmi.csv', index=False)
tuji_filmi

,naslov_nom,naslov_orig,leto,kategorija,drzava,jezik
0,Marie-Louise,Marie-Louise,1945,Writing,Switzerland,German
1,Gate of Hell,地獄門,1954,Costume Design,Japan,Japanese
2,The Red Balloon,Le ballon rouge,1956,Writing,France,French
3,Serengeti Shall Not Die,Serengeti darf nicht sterben,1959,Documentary Feature,Germany,German
4,Serengeti Shall Not Die,Serengeti darf nicht sterben,1959,Documentary Feature,Germany,German
...,...,...,...,...,...,...
78,20 Days in Mariupol,20 днів у Маріуполі,2023,Documentary Feature,Ukraine,Ukrainian
79,20 Days in Mariupol,20 днів у Маріуполі,2023,Documentary Feature,Ukraine,English
80,20 Days in Mariupol,20 днів у Маріуполі,2023,Documentary Feature,Ukraine,Russian
81,Anatomy of a Fall,Anatomie d'une chute,2023,Original Screenplay,France,French


## List of actors with Academy Award nominations

In [293]:
url5 = 'https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20actors%20with%20Academy%20Award%20nominations%20-%20Wikipedia.html'
tabela5 = pd.read_html(url5)[1]
tabela5.rename(columns={'Actor':'ime', 'Born':'leto_rojstva', 'Died':'leto_smrti', 'Nominations':'nominacija', 'First winning film role or first nomination (also see list of all nominated roles)':'prvi_film'},inplace=True)
tabela5.columns.values[1] = 'spol'

def obdelaj_vrstico(vrstica):
    if len(vrstica) == 1:
        if vrstica == 'S':
            return [('Glavna',False,0),('Glavna',True,0),('Stranska',False,'izracunaj'),('Stranska',True,'izracunaj')]
        elif vrstica == 'L':
            return [('Glavna',False,'izracunaj'),('Glavna',True,'izracunaj'),('Stranska',False,0),('Stranska',True,0)]
    else:
        deli = vrstica.split(':')
        glavni_del = deli[0]
        stranski_del = deli[1]
        if len(glavni_del) == 2:
            glavni_del += '0'
        if len(stranski_del) == 2:
            stranski_del += '0'
        #št nominacij in zmag
        nom_glavni = int(glavni_del.split('L')[0])
        os_glavni = int(glavni_del.split('L')[1])
        false_glavni = nom_glavni-os_glavni
        nom_stranski = int(stranski_del.split('S')[0])
        os_stranski = int(stranski_del.split('S')[1])
        false_stranski = nom_stranski-os_stranski
        return [('Glavna',False,false_glavni),('Glavna',True,os_glavni),('Stranska',False,false_stranski),('Stranska',True,os_stranski)]

# Dodajanje stolpca 'Rezultati', ki ga generiramo s funkcijo obdelaj_vrstico
tabela5['Rezultati'] = tabela5['Lead and sup- porting details'].map(obdelaj_vrstico)

# Razširitev v DataFrame z razbijanjem 'Rezultati'
razsirjeni_podatki = pd.DataFrame(
    [(ime, nominacija, uspeh, stevilo) 
     for ime, vrstica in zip(tabela5['ime'], tabela5['Rezultati']) 
     for nominacija, uspeh, stevilo in vrstica],
    columns=['ime', 'nominacija', 'oskar', 'stevilo']
)

# Priključitev ostalih stolpcev
razsirjeni_podatki = razsirjeni_podatki.merge(
    tabela5.drop(columns=['Rezultati', 'Lead and sup- porting details']),
    left_on='ime',
    right_on='ime',
    how='left'
)


razsirjeni_podatki['Nomina- tions'] = pd.to_numeric(razsirjeni_podatki['Nomina- tions'], errors='coerce')
razsirjeni_podatki['Wins'] = pd.to_numeric(razsirjeni_podatki['Wins'], errors='coerce')

# Funkcija za nastavitev vrednosti v stolpcu 'stevilo'
def nastavi_stevilo_vrstic(vrstica):
    if vrstica['stevilo'] == 'izracunaj':
        if vrstica['oskar'] == True:
            return vrstica['Wins']  # Število zmag
        else:
            return vrstica['Nomina- tions'] - vrstica['Wins']  # Število neuspešnih nominacij
    return vrstica['stevilo']

# Posodobitev stolpca 'stevilo' za vsako vrstico
razsirjeni_podatki['stevilo'] = razsirjeni_podatki.apply(nastavi_stevilo_vrstic, axis=1)
razsirjeni_podatki['stevilo'] = razsirjeni_podatki['stevilo'].fillna(0).astype(int)

igralci = razsirjeni_podatki[['ime','spol','leto_rojstva','leto_smrti','nominacija','oskar','stevilo','prvi_film']]

igralci.to_csv(r'C:\Users\Aneja\Documents\pavp\igralci.csv', index=False)
igralci

,ime,spol,leto_rojstva,leto_smrti,nominacija,oskar,stevilo,prvi_film
0,Barkhad Abdi,M,1985,~,Glavna,False,0,Captain Phillips
1,Barkhad Abdi,M,1985,~,Glavna,True,0,Captain Phillips
2,Barkhad Abdi,M,1985,~,Stranska,False,1,Captain Phillips
3,Barkhad Abdi,M,1985,~,Stranska,True,0,Captain Phillips
4,F. Murray Abraham,M,1939,~,Glavna,False,0,Amadeus
...,...,...,...,...,...,...,...,...
3915,Renée Zellweger,F,1969,~,Stranska,True,1,Judy [w 44]
3916,Catherine Zeta-Jones,F,1969,~,Glavna,False,0,Chicago
3917,Catherine Zeta-Jones,F,1969,~,Glavna,True,0,Chicago
3918,Catherine Zeta-Jones,F,1969,~,Stranska,False,0,Chicago
